In [18]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import pandas as pd
import numpy as np
import os
import timeit
# Notes on Multicore processsing
# First define the function that you will be dropping into the pool
# Next declare a function that can take thr number of partit

ts_df = pd.DataFrame(np.random.random(size=(365, 3000)))



def feature_calculation(df):
    # create DataFrame and populate with stdDev
    result = pd.DataFrame(df.std(axis=0))
    result.columns = ["stdDev"]
    
    # mean
    result["mean"] = df.mean(axis=0)

    # percentiles
    for i in [0.1, 0.25, 0.5, 0.75, 0.9]:
        result[str(int(i*100)) + "perc"] = df.quantile(q=i)

    # percentile differences / amplitudes
    result["diff_90perc10perc"] = (result["10perc"] - result["90perc"])
    result["diff_75perc25perc"] = (result["75perc"] - result["25perc"])

    # percentiles of lagged time-series
    for lag in [10, 20, 30, 40, 50]:
        for i in [0.1, 0.25, 0.5, 0.75, 0.9]:
            result["lag" + str(lag) + "_" + str(int(i*100)) + "perc"] = (df - df.shift(lag)).quantile(q=i)

    # fft
    df_fft = np.fft.fft(df, axis=0)  # fourier transform only along time axis
    result["fft_angle_mean"] = np.mean(np.angle(df_fft, deg=True), axis=0)
    result["fft_angle_min"] = np.min(np.angle(df_fft, deg=True), axis=0)
    result["fft_angle_max"] = np.max(np.angle(df_fft, deg=True), axis=0)
    
    return result


def parallel_feature_calculation_concurent_processpool(df, partitions=10, processes=4):
    # calculate features in paralell by splitting the dataframe into partitions and using paralell processes
    
    df_split = np.array_split(df, partitions, axis=1)  # split dataframe into partitions column wise
    
    with ProcessPoolExecutor(processes) as pool:        
        df = pd.concat(pool.map(feature_calculation, df_split))
    
    return df



from multiprocessing import Pool

def parallel_feature_calculation_multiprocess(df, partitions=10, processes=4):
    # calculate features in parallel by splitting the dataframe into partitions and using parallel processes
    
    pool = Pool(processes)
    
    df_split = np.array_split(df, partitions, axis=1)  # split dataframe into partitions column wise
    
    df = pd.concat(pool.map(feature_calculation, df_split))
    pool.close()
    pool.join()
    
    return df


In [19]:


%timeit feature_calculation(ts_df)

14.2 s ± 2.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:

%timeit -n 1 -r 1 parallel_feature_calculation_concurent_processpool(ts_df, partitions=100, processes=4)



13.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:

%timeit -n 1 -r 1 parallel_feature_calculation_multiprocess(ts_df, partitions=100, processes=4)


11 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
